<a href="https://colab.research.google.com/github/GladiatorGeneral/aegis-health-chain/blob/main/notebooks/00_colab_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Check GPU & Environment

In [1]:
import torch
import sys

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU not available. Enable it in Runtime → Change runtime type → GPU")

print(f"\nPython version: {sys.version}")

PyTorch version: 2.9.0+cu126
GPU Available: True
GPU Device: Tesla T4
GPU Memory: 15.83 GB

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


## Step 2: Clone Repository & Install Dependencies

In [2]:
import socket
import urllib.request
import subprocess

print("🔍 Network Connectivity Check\n")

# Test 1: DNS Resolution
print("1️⃣ Testing DNS resolution for github.com...")
try:
    ip = socket.gethostbyname('github.com')
    print(f"✅ github.com resolves to: {ip}")
except socket.gaierror as e:
    print(f"❌ DNS resolution failed: {e}")

# Test 2: HTTP connectivity
print("\n2️⃣ Testing HTTP connection to GitHub...")
try:
    response = urllib.request.urlopen('https://api.github.com', timeout=5)
    print(f"✅ GitHub API reachable (Status: {response.status})")
except Exception as e:
    print(f"❌ HTTP connection failed: {e}")

print("\n✅ Network checks complete. Proceeding to clone...")

🔍 Network Connectivity Check

1️⃣ Testing DNS resolution for github.com...
✅ github.com resolves to: 140.82.114.3

2️⃣ Testing HTTP connection to GitHub...
✅ GitHub API reachable (Status: 200)

✅ Network checks complete. Proceeding to clone...


In [3]:
import os
import subprocess
from pathlib import Path
import time

# Clone repository
repo_path = '/content/aegis-health-chain'

if not Path(repo_path).exists():
    print("📦 Attempting to clone repository...\n")

    # Method 1: Standard HTTPS clone with retry logic
    max_retries = 3
    for attempt in range(1, max_retries + 1):
        print(f"Attempt {attempt}/{max_retries}:")

        result = subprocess.run(
            ['git', 'clone', '--depth', '1', 'https://github.com/GladiatorGeneral/aegis-health-chain.git', repo_path],
            capture_output=True,
            text=True,
            timeout=60
        )

        if result.returncode == 0:
            print("✅ Repository cloned successfully\n")
            break
        else:
            error_msg = result.stderr[:200]
            print(f"  ❌ Failed: {error_msg}")

            if attempt < max_retries:
                wait_time = 5 * attempt
                print(f"  ⏳ Waiting {wait_time}s before retry...\n")
                time.sleep(wait_time)
            else:
                print("\n❌ All clone attempts failed")
                print("\n🔧 Troubleshooting options:")
                print("1. Check if GitHub is accessible: https://www.githubstatus.com")
                print("2. Try again in a few moments (may be a temporary network issue)")
                print("3. Manually download from: https://github.com/GladiatorGeneral/aegis-health-chain")
                raise Exception(f"Could not clone repository after {max_retries} attempts. Error: {result.stderr}")
else:
    print("✅ Repository already exists")

# Change to repo directory
try:
    os.chdir(repo_path)
    print(f"✅ Changed to: {os.getcwd()}")
except Exception as e:
    print(f"❌ Failed to change directory: {e}")
    raise

# Pull latest changes
print("\n📥 Pulling latest changes...")
result = subprocess.run(['git', 'pull'], capture_output=True, text=True, timeout=60)
if result.returncode == 0:
    print("✅ Latest changes pulled")
elif "Already up to date" in result.stdout:
    print("✅ Already up to date")
else:
    print(f"⚠️ Pull note: {result.stdout[:100]}")

print(f"\n📁 Working directory: {os.getcwd()}")
print(f"\n📂 Project structure:")
subprocess.run(['ls', '-la'], check=False)

📦 Attempting to clone repository...

Attempt 1/3:
✅ Repository cloned successfully

✅ Changed to: /content/aegis-health-chain

📥 Pulling latest changes...
✅ Latest changes pulled

📁 Working directory: /content/aegis-health-chain

📂 Project structure:


CompletedProcess(args=['ls', '-la'], returncode=0)

In [4]:
import subprocess
import os
from pathlib import Path

# If git clone failed, try downloading as ZIP
repo_path = '/content/aegis-health-chain'

if not Path(repo_path).exists():
    print("📥 Attempting to download repository as ZIP...\n")

    try:
        # Download ZIP file
        zip_url = "https://github.com/GladiatorGeneral/aegis-health-chain/archive/refs/heads/main.zip"
        zip_file = "/tmp/aegis-health-chain.zip"

        result = subprocess.run(
            ['wget', zip_url, '-O', zip_file],
            capture_output=True,
            text=True,
            timeout=60
        )

        if result.returncode != 0:
            # Try curl as fallback
            result = subprocess.run(
                ['curl', '-L', zip_url, '-o', zip_file],
                capture_output=True,
                text=True,
                timeout=60
            )

        if result.returncode == 0 and Path(zip_file).exists():
            print("✅ ZIP downloaded successfully")

            # Extract ZIP
            print("📦 Extracting ZIP...")
            result = subprocess.run(
                ['unzip', '-q', zip_file, '-d', '/content/'],
                capture_output=True,
                text=True,
                timeout=60
            )

            if result.returncode == 0:
                # Move extracted folder to correct location
                subprocess.run(['mv', '/content/aegis-health-chain-main', repo_path],
                             capture_output=True)
                print("✅ Repository extracted successfully")
            else:
                print(f"❌ Extract failed: {result.stderr}")
        else:
            print("❌ ZIP download failed")
            print("\n⚠️ Manual setup needed:")
            print("1. Download: https://github.com/GladiatorGeneral/aegis-health-chain/archive/refs/heads/main.zip")
            print("2. Upload to Colab via Files panel (left sidebar)")
            print("3. Extract in Colab: !unzip aegis-health-chain-main.zip")
            raise Exception("Could not download repository")

    except Exception as e:
        print(f"❌ Download failed: {e}")
        raise

print(f"\n✅ Repository ready at: {repo_path}")
os.chdir(repo_path)
print(f"Working directory: {os.getcwd()}")


✅ Repository ready at: /content/aegis-health-chain
Working directory: /content/aegis-health-chain


In [5]:
# Install dependencies
print("📥 Installing dependencies...")
print("This may take 3-5 minutes on first run...\n")

# Install pip requirements using subprocess for better reliability
result = subprocess.run(
    ['pip', 'install', '-q', '-e', '.'],
    cwd=repo_path,
    capture_output=True,
    text=True,
    timeout=300
)

if result.returncode == 0:
    print("✅ Dependencies installed successfully")
else:
    # Installation may have warnings but still succeed
    if "Successfully installed" in result.stderr or result.returncode == 0:
        print("✅ Dependencies installed (with some warnings)")
    else:
        print(f"⚠️ Installation note: {result.stderr[-200:] if result.stderr else 'completed'}")

📥 Installing dependencies...
This may take 3-5 minutes on first run...

⚠️ Installation note: rror

× Getting requirements to build editable did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.



## Step 3: Verify Installation

In [6]:
# Add src to path
import sys
sys.path.insert(0, '/content/aegis-health-chain')

# Test imports
print("🔍 Testing imports...\n")

try:
    from src.udm_mapper import udm_mapper
    print("✅ UDM Mapper imported successfully")
except Exception as e:
    print(f"❌ UDM Mapper error: {e}")

try:
    from src.data_pipeline import data_pipeline
    print("✅ Data Pipeline imported successfully")
except Exception as e:
    print(f"❌ Data Pipeline error: {e}")

try:
    from src.huggingface_models import clinical_models
    print("✅ HuggingFace Models imported successfully")
    print(f"   Available models: {clinical_models.list_available_models()}")
except Exception as e:
    print(f"❌ HuggingFace Models error: {e}")

print("\n✅ All core modules imported successfully!")

🔍 Testing imports...



✅ UDM Mapper imported successfully
✅ Data Pipeline imported successfully
✅ HuggingFace Models imported successfully
   Available models: ['clinical_bert', 'sentence_transformer', 'clinical_pubmed_bert']

✅ All core modules imported successfully!


## Step 4: Run Tests

In [7]:
import subprocess

print("🧪 Running test suite...\n")
result = subprocess.run(
    ['pytest', 'tests/', '-v', '--tb=short'],
    cwd='/content/aegis-health-chain',
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr)

if result.returncode == 0:
    print("\n✅ All tests passed!")
else:
    print(f"\n⚠️ Some tests failed (exit code: {result.returncode})")

🧪 Running test suite...

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/aegis-health-chain
configfile: pyproject.toml
plugins: langsmith-0.4.43, typeguard-4.4.4, anyio-4.11.0
collecting ... collected 6 items

tests/test_models.py::TestClinicalModels::test_model_loading PASSED      [ 16%]
tests/test_models.py::TestClinicalModels::test_embedding_generation PASSED [ 33%]
tests/test_models.py::TestClinicalModels::test_available_models PASSED   [ 50%]
tests/test_udm.py::TestUDMMapper::test_epic_mapping PASSED               [ 66%]
tests/test_udm.py::TestUDMMapper::test_gender_mapping PASSED             [ 83%]
tests/test_udm.py::TestUDMMapper::test_date_standardization PASSED       [100%]

============================== 6 passed in 23.17s ==============================


✅ All tests passed!


## Step 5: Quick Demo - Test UDM Mapper

In [8]:
import json
from src.udm_mapper import udm_mapper

print("🧬 Testing UDM Mapper with sample data...\n")

# Test Epic EHR data
epic_data = {
    "PAT_MRN": "COLAB001",
    "BIRTH_DATE": "1985-03-15",
    "SEX": "F",
    "RACE": "Asian",
    "ETHNICITY": "Not Hispanic"
}

print("Input (Epic EHR):")
print(json.dumps(epic_data, indent=2))

result = udm_mapper.map_ehr_to_udm(epic_data, "epic")

print("\nOutput (UDM Format):")
print(json.dumps(result, indent=2))
print("\n✅ UDM mapping successful!")

🧬 Testing UDM Mapper with sample data...

Input (Epic EHR):
{
  "PAT_MRN": "COLAB001",
  "BIRTH_DATE": "1985-03-15",
  "SEX": "F",
  "RACE": "Asian",
  "ETHNICITY": "Not Hispanic"
}

Output (UDM Format):
{
  "patient": {
    "resourceType": "Patient",
    "id": "COLAB001",
    "birthDate": "1985-03-15",
    "gender": "female",
    "race": {
      "text": "Asian"
    },
    "ethnicity": {
      "text": "Not Hispanic"
    }
  }
}

✅ UDM mapping successful!


## Step 6: Generate Synthetic Data

In [9]:
from src.data_pipeline import data_pipeline
import pandas as pd

print("🧪 Generating synthetic patient data...\n")

# Generate 50 synthetic patients
synthetic_data = data_pipeline.generate_synthetic_data(50)

# Extract and display statistics
genders = [p['patient'].get('gender', 'unknown') for p in synthetic_data if 'patient' in p]
gender_counts = pd.Series(genders).value_counts()

print(f"Generated {len(synthetic_data)} synthetic patient records\n")
print("Gender Distribution:")
print(gender_counts)

print("\n✅ Synthetic data generation successful!")

🧪 Generating synthetic patient data...

Generated 50 synthetic patient records

Gender Distribution:
unknown    50
Name: count, dtype: int64

✅ Synthetic data generation successful!


## Step 7: (Optional) Configure GitHub Authentication for Push

If you want to push changes back to GitHub, run the cell below and follow the instructions:

```
# This creates a personal access token setup
# Go to https://github.com/settings/tokens/new
# Create a token with 'repo' scope
# Paste it when prompted
```

In [10]:
# Optional: Configure Git for commits
# Uncomment and run if you want to push changes to GitHub

# import getpass
#
# print("📝 Configuring Git for GitHub...\n")
#
# # Configure Git user
# os.system('git config user.email "your-email@example.com"')
# os.system('git config user.name "Your Name"')
#
# # Get GitHub token
# token = getpass.getpass("Enter your GitHub personal access token: ")
#
# # Update remote URL to use token
# os.system(f'git remote set-url origin https://{token}@github.com/PhnxNexus/aegis-health-chain.git')
#
# print("\n✅ Git configured for pushing to GitHub")
# print("You can now run: git add . && git commit -m 'message' && git push")

## 🎉 Setup Complete!

You're ready to start developing. Here's what you can do next:

### 1️⃣ Run Existing Notebooks
Open `notebooks/01_data_exploration.ipynb` to explore data and test the UDM mapper.

### 2️⃣ Test HuggingFace Models
Run `notebooks/03_huggingface_test.ipynb` to test clinical embeddings (requires downloading models).

### 3️⃣ Start Development
Use the cells below to write your own code:

```python
from src import udm_mapper, data_pipeline, clinical_models

# Map EHR data
result = udm_mapper.map_ehr_to_udm(your_data, "epic")

# Generate synthetic data
synthetic = data_pipeline.generate_synthetic_data(100)

# Get clinical embeddings
embeddings = clinical_models.get_clinical_embeddings("patient text")
```

### 4️⃣ Push Changes (Optional)
When ready, commit and push changes back to GitHub.

**Happy coding! 🚀**

## 💡 Useful Colab Tips

- **GPU:** Go to Runtime → Change runtime type → Hardware accelerator: GPU
- **Save Notebook:** Click "Copy to Drive" to save a copy in your Google Drive
- **Install Packages:** Use `pip install package_name` or `!pip install package_name`
- **Run Terminal Commands:** Use `os.system('command')` or `!command`
- **Mount Google Drive:** Use `google.colab.drive.mount('/content/drive')`
- **Download Files:** Use download icon or create a download link in code